In [15]:
# -*- coding: utf-8 -*-
"""entity-sentiment-analysis-model-create.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1foQEI0H7nccEd4ah2fynAU9Kf4MPBuXZ
"""

!pip install datasets transformers

!apt install git-lfs

!git lfs install

!git config --global user.email "aliemre9270@gmail.com"
!git config --global user.name "WhiteAngels"

!git config --global credential.helper store

from huggingface_hub import notebook_login
notebook_login()

from google.colab import drive
drive.mount('/content/gdrive')

file_name = "test.csv"
path = "/content/gdrive/MyDrive/Colab Notebooks/input/"

from datasets import load_dataset
dataset = load_dataset("csv", data_files= path+file_name)

dataset

dataset['train'][:2]

reviews_sample = dataset["train"].shuffle(seed=42).select(range(4000))

reviews_sample

reviews_sample = reviews_sample.rename_column(
    original_column_name="text", new_column_name="review"
)
reviews_sample

# Yorum uzunluklarını hesaplayan bir fonksiyon
def compute_review_length(example):
    example['review_length'] = len(example['review'].split())
    return example

import pandas as pd
from datasets import Dataset
csv_path = '/content/gdrive/MyDrive/Colab Notebooks/input/test.csv'
df = pd.read_csv(csv_path)

# Dataset oluşturun
dataset = Dataset.from_pandas(df.rename(columns={'text': 'review'})) # Rename column in df

# Yorum uzunluklarını hesaplayın
dataset = dataset.map(compute_review_length)

# Çıktıyı formatlı bir şekilde göster
for example in dataset:
    print(f"Review: {example['review']}")
    print(f"Label: {example['label']}\n")

# İşlenmiş veriyi diske kaydedin
output_path = '/content/gdrive/MyDrive/Colab Notebooks/output/düzenlenmişveri'
dataset.save_to_disk(output_path)

# Çıktıyı formatlı bir şekilde göster
for i in range(5):
    example = dataset[i]  # Get the i-th element as a dictionary
    print(f"Review: {example['review']}")
    print(f"Label: {example['label']}\n")

dataset_ek = load_dataset('ctoraman/atis-ner-turkish', split='train')

from transformers import BertTokenizer, BertForTokenClassification
# Model ve tokenizer'ı yükleyin
model_name = 'akdeniz27/bert-base-turkish-cased-ner'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

from datasets import load_dataset, Dataset
dataset = Dataset.from_pandas(df)
dataset_ek= Dataset.from_pandas(df)

from transformers import pipeline
ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy='simple')

def apply_ner(example):
    review = example['text']
    ner_results = ner_pipeline(review)
    # NER sonuçlarını örnek veri yapısına ekle
    example['entities'] = [result['entity'] for result in ner_results]
    example['scores'] = [result['score'] for result in ner_results]  # NER sonucundaki skorları ekler
    return example

print(dataset.column_names)
print(dataset_ek.column_names)

def apply_ner(example):
    review = example['text']
    ner_results = ner_pipeline(review)

    # NER sonuçlarını uygun formatta hazırlama
    entities = [result['entity_group'] for result in ner_results]
    scores = [result['score'] for result in ner_results]

    # Entity listesi ve skorları formatla
    example['entity_list'] = entities  # NER tanımlarının listesi
    example['entity'] = entities       # Entities için kullanılacak liste (ihtiyaca göre özelleştirilebilir)
    example['score'] = scores          # NER sonuçlarındaki skorlar

    # Sentiment bilgisi
    sentiment = example.get('label', 'Unknown')
    example['sentiment'] = sentiment

    return example

subset = dataset.select(range(1000))

dataset_with_entities = subset.map(apply_ner)

# İşlenmiş veriyi diske kaydedin
output_path = '/content/gdrive/MyDrive/Colab Notebooks/output/reviews_with_entities'
dataset_with_entities.save_to_disk(output_path)

# İlk 5 veriyi formatlı şekilde göster
for i in range(5):
    example = dataset_with_entities[i]
    review = example['text']
    entity_list = example.get('entity_list', [])
    score = example.get('score', [])
    sentiment = example.get('sentiment', 'Unknown')

    print(f"Review: {review}")

    if entity_list:
        for entity, score in zip(entity_list, score):
            print(f"Entity: {entity}, Score: {score}")
    else:
        print("Entities data are not available.")

    print(f"Overall Sentiment: {sentiment}\n")

!pip install -U textblob
!python -m textblob.download_corpora
!pip install stanza transformers

import pandas as pd
import re

# Temizleme fonksiyonu
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Birden fazla boşluğu tek boşluğa indirgeme
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Kullanıcı etiketlerini kaldırma
    text = re.sub(r'http\S+', '', text)  # URL'leri kaldırma
    text = re.sub(r'[^\w\s]', '', text)  # Noktalama işaretlerini kaldırma
    text = text.strip()  # Baş ve sondaki boşlukları kaldırma
    return text

# Pozitif ve negatif kelime listeleri
positive_words = ["iyi", "güzel", "mükemmel", "harika", "fantastik", "olumlu", "beğendim"]
negative_words = ["kötü", "berbat", "felaket", "rezalet", "olumsuz", "sevmedim"]

# Veri yükleme
csv_path = "/content/gdrive/MyDrive/Colab Notebooks/input/test.csv"
df = pd.read_csv(csv_path)

# Veriyi temizleme ve temizlenmiş metni ekleme
df['cleaned_text'] = df['text'].apply(clean_text)

# Temizlenmiş veriyi ve label kolonunu gösterme
print(df[['text', 'label', 'cleaned_text', 'label']].head())

from textblob import TextBlob
from collections import defaultdict

# Daha akıllı entity tanımlama fonksiyonu
def smart_extract_entities(text):
    entities = []
    current_entity = []
    for word in text.split():
        if word[0].isupper() or word.startswith("@"):
            current_entity.append(word)
        else:
            if current_entity:
                entities.append(' '.join(current_entity))
                current_entity = []
    if current_entity:
        entities.append(' '.join(current_entity))
    return entities

# Sentiment çıkarma fonksiyonu
def extract_sentiment(text, entity):
    blob = TextBlob(text)
    sentences = blob.sentences
    entity_sentiment = "Notr"
    for sentence in sentences:
        if entity in sentence:
            polarity = sentence.sentiment.polarity
            if polarity > 0:
                entity_sentiment = "Pozitif"
            elif polarity < 0:
                entity_sentiment = "Negatif"
            else:
                entity_sentiment = "Notr"
    return entity_sentiment

# Entity ve sentiment çıkarımı
df['entities'] = df['cleaned_text'].apply(smart_extract_entities)
df['results'] = df.apply(lambda x: [{"entity": entity, "sentiment": extract_sentiment(x['cleaned_text'], entity)} for entity in x['entities']], axis=1)
for index, row in df.head(20).iterrows():
    print(f"Girdi: {row['text']}")
    print(f"Çıktı: {{\n 'entity_list': {row['entities']},\n 'results': [")
    for result in row['results']:
        print(f"  {{'entity': '{result['entity']}', 'sentiment': '{result['sentiment']}'}}")
    print(" ]\n}")
print(df.head(20))


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Git LFS initialized.


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Map:   0%|          | 0/48965 [00:00<?, ? examples/s]

Görüntülenen çıkış son 5000 satıra kısaltıldı.
Label: Positive

Review: Gürcistan Yazarlar Birliği Acara şubesi başkanı oldu .
Label: Notr

Review: ürün gayet güzel ve kaliteli, keşke mumları da birlikte verilseydi.
Label: Positive

Review: Bu türün ilk örnekleri Ahmet Paşa tarafından verilmiştir .
Label: Notr

Review: Ayrıca Oh!
Label: Notr

Review:  Az önce izledim filmi Çok hoş bir Duygusal komedi filmiydi. izlerken sıkılmıyor gayet hoş ve eylenceli vakit geçirebiliyorsunuz. İzlediğim en iyi duygusal komedi filmlerinden birisiydi :) 
Label: Positive

Review: eşime hediye olarak aldım. kargoya zamanında verildi ve hızlı bir şekilde elime ulaştı.üründen memnunum.
Label: Positive

Review: uygun fiyatıyla almaya değer, iş görüyor, sesli ve görüntülü ekranı ile uydunuzu ayarlayabilirsiniz.
Label: Positive

Review: Ayrıca kulübün profesyonel liglerde mücadele eden bir diğer takımı da erkek hentbol takımıdır .
Label: Notr

Review: Yazar gelecekten gelir ve Şehrazad’a ve Bin Bir Gece Masall

Saving the dataset (0/1 shards):   0%|          | 0/48965 [00:00<?, ? examples/s]

Review: Kral akbaba dikkat çekici renklere sahiptir .
Label: Notr

Review:  ısrarla korkutmayı başarıyor. sanki korku çok uzun ve bitmiyor. zira film bitiyor ama müzik halâ uyarmaya devam ediyor. 9... 
Label: Positive

Review: Neşe ve Üzüntü köprünün kırılmaya başlamasıyla geri dönerler .
Label: Notr

Review: i phone 5 ten sonra gene 4'' ekranı tercih ettim. telefon mükemmel. ertesi gün elime ulaştı. teşekkürler
Label: Positive

Review: Beşinci sezonda diziye yeni oyuncular katıldı .
Label: Notr

['text', 'label']
['text', 'label']


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Review: Kral akbaba dikkat çekici renklere sahiptir .
Entities data are not available.
Overall Sentiment: Notr

Review:  ısrarla korkutmayı başarıyor. sanki korku çok uzun ve bitmiyor. zira film bitiyor ama müzik halâ uyarmaya devam ediyor. 9... 
Entities data are not available.
Overall Sentiment: Positive

Review: Neşe ve Üzüntü köprünün kırılmaya başlamasıyla geri dönerler .
Entity: PER, Score: 0.9805818200111389
Entity: PER, Score: 0.987076461315155
Entity: PER, Score: 0.9296213984489441
Entity: PER, Score: 0.9243172407150269
Entity: PER, Score: 0.9746062755584717
Overall Sentiment: Notr

Review: i phone 5 ten sonra gene 4'' ekranı tercih ettim. telefon mükemmel. ertesi gün elime ulaştı. teşekkürler
Entities data are not available.
Overall Sentiment: Positive

Review: Beşinci sezonda diziye yeni oyuncular katıldı .
Entities data are not available.
Overall Sentiment: Notr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: textblob

In [31]:
import torch
import torch.nn as nn
from transformers import BertTokenizerFast, BertModel, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from datasets import load_metric

# Create custom tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

class MultitaskModel(nn.Module):
    def __init__(self, model_name, num_labels_ner, num_labels_sa):
        super(MultitaskModel, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.ner_head = nn.Linear(self.bert.config.hidden_size, num_labels_ner)
        self.sa_head = nn.Linear(self.bert.config.hidden_size, num_labels_sa)

    def forward(self, input_ids, attention_mask, token_type_ids, ner_labels=None, sa_labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[0]
        pooled_output = outputs[1]

        ner_logits = self.ner_head(sequence_output)
        sa_logits = self.sa_head(pooled_output)

        outputs = (ner_logits, sa_logits)

        if ner_labels is not None and sa_labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            ner_loss = loss_fct(ner_logits.view(-1, self.ner_head.out_features), ner_labels.view(-1))
            sa_loss = loss_fct(sa_logits.view(-1, self.sa_head.out_features), sa_labels.view(-1))
            loss = ner_loss + sa_loss
            outputs = (loss,) + outputs

        return outputs

# Initialize the model
model_name = 'bert-base-uncased'
num_labels_ner = 3  # Adjust based on your NER labels
num_labels_sa = 3   # Adjust based on your sentiment labels
model = MultitaskModel(model_name, num_labels_ner, num_labels_sa)


In [39]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizerFast, BertForTokenClassification, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
# Assuming you are using the datasets library
from datasets import load_dataset

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased') # Initialize tokenizer here

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(df['cleaned_text'], truncation=True, is_split_into_words=True)
    labels_ner = []
    labels_sa = []
    for i, label in enumerate(examples['label']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids_ner = [-100 if i is None else label for i in word_ids]
        labels_ner.append(label_ids_ner)
        labels_sa.append(label)  # Assuming `label` contains sentiment labels
    tokenized_inputs["labels_ner"] = labels_ner
    tokenized_inputs["labels_sa"] = labels_sa
    return tokenized_inputs


In [40]:
import numpy as np
from datasets import load_metric

# Load metrics
metric_ner = load_metric("seqeval")
metric_sa = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)
    labels = labels

    # Compute F1 for NER
    ner_results = metric_ner.compute(predictions=predictions.tolist(), references=labels.tolist())

    # Compute Accuracy for Sentiment Analysis
    sa_results = metric_sa.compute(predictions=predictions.argmax(axis=-1).tolist(), references=labels.tolist())

    return {
        'ner_f1': ner_results['f1'],
        'sa_accuracy': sa_results['accuracy']
    }


In [22]:
!pip install seqeval
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=1232a58a408266d5b4d82f514b4fc82e54d7bbc96eb3b8ed377808e68f3d5b0f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00


In [50]:
# Load metrics
metric_ner = load_metric("seqeval")
metric_sa = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions_ner = np.argmax(predictions[0], axis=2)
    labels_ner = labels[0]
    predictions_sa = np.argmax(predictions[1], axis=1)
    labels_sa = labels[1]

    # Compute F1 for NER
    ner_results = metric_ner.compute(predictions=predictions_ner.tolist(), references=labels_ner.tolist())

    # Compute Accuracy for Sentiment Analysis
    sa_results = metric_sa.compute(predictions=predictions_sa.tolist(), references=labels_sa.tolist())

    return {
        'ner_f1': ner_results['f1'],
        'sa_accuracy': sa_results['accuracy']
    }


In [53]:
def tokenize_and_align_labels(examples):
    # Tokenize the texts
    tokenized_inputs = tokenizer(df['cleaned_text'], padding=True, truncation=True, is_split_into_words=False, return_tensors="pt")

    # Assuming 'label' is already in the correct format and aligns with the tokenized output
    labels = examples['label']

    # Ensure that 'label' has the same length as the tokenized text
    # Adjust this if you have special handling for your labels
    tokenized_inputs['labels'] = labels

    return tokenized_inputs
print(dataset.column_names)

# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)
# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/gdrive/MyDrive/Colab Notebooks/output',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='/content/gdrive/MyDrive/Colab Notebooks/output/logs',
    logging_steps=200,
)

# Initialize Trainer
class MultitaskTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels_ner = inputs.pop("labels_ner")
        labels_sa = inputs.pop("labels_sa")
        outputs = model(**inputs, ner_labels=labels_ner, sa_labels=labels_sa)
        loss = outputs[0]
        return (loss, outputs) if return_outputs else loss

trainer = MultitaskTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()


{'train': ['text', 'label', 'dataset'], 'test': ['text', 'label', 'dataset']}


Map:   0%|          | 0/440679 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [48]:
# Example list of texts
batch_texts = ["This is the first sentence.", "This is the second sentence."]

# Tokenize the batch of texts
encoded_batch = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")

print("Encoded batch examples:", encoded_batch)


Encoded batch examples: {'input_ids': tensor([[ 101, 2023, 2003, 1996, 2034, 6251, 1012,  102],
        [ 101, 2023, 2003, 1996, 2117, 6251, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


In [47]:
from transformers import BertTokenizerFast

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Example single text
single_text = "This is a single example sentence."

# Tokenize the single text
encoded_single = tokenizer(single_text, padding=True, truncation=True, return_tensors="pt")

print("Encoded single example:", encoded_single)


Encoded single example: {'input_ids': tensor([[ 101, 2023, 2003, 1037, 2309, 2742, 6251, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# Save model and tokenizer
model.save_pretrained('/content/gdrive/MyDrive/Colab Notebooks/output/model')
tokenizer.save_pretrained('/content/gdrive/MyDrive/Colab Notebooks/output/tokenizer')


In [ ]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

def format_entities(entities):
    return [{'entity': entity['entity'], 'score': entity['score']} for entity in entities]

# Example inference
text = "Your example text here"
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    ner_logits, sa_logits = outputs[:2]
    ner_predictions = torch.argmax(ner_logits, dim=2).squeeze().tolist()
    sa_predictions = torch.argmax(sa_logits, dim=1).tolist()

# Print results
print("Entities:", format_entities(ner_predictions))
print("Sentiment Analysis:", sa_predictions)
